In [63]:
from keras.layers          import Lambda, Input, Dense, GRU, LSTM, RepeatVector
from keras.models          import Model
from keras.layers.core     import Flatten
from keras.callbacks       import LambdaCallback
from keras.optimizers      import SGD, RMSprop, Adam
from keras.layers.wrappers import Bidirectional as Bi
from keras.layers.wrappers import TimeDistributed as TD
from keras.layers          import merge, multiply
from keras.regularizers    import l2
from keras.layers.core     import Reshape
from keras.layers.normalization import BatchNormalization as BN
import keras.backend as K
import numpy as np
import random
import sys
import pickle
import glob
import copy
import os
import re
import MeCab
import keras
import time

In [104]:
timesteps   = 50
inputs      = Input(shape=(timesteps, 128))
encoded     = LSTM(512)(inputs)
inputs_a    = inputs
inputs_a    = Dense(2048)(inputs_a)
inputs_a    = BN()(inputs_a)
a_vector    = Dense(512, activation='softmax')(Flatten()(inputs_a))
mul         = multiply([encoded, a_vector])
encoder     = Model(inputs, mul)

x           = RepeatVector(timesteps)(mul)
x           = Bi(LSTM(512, return_sequences=True))(x)
decoded     = TD(Dense(128, activation='softmax'))(x)

autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer=Adam(), loss='categorical_crossentropy')

buff = None

In [105]:
def callbacks(epoch, logs):
  global buff
  buff = copy.copy(logs)
  print("epoch" ,epoch)
  print("logs", logs)

In [107]:
def train():
  c_i = pickle.loads( open("dict_pkl/c_i_formula.pkl", "rb").read() )
  c_i2 = pickle.loads( open("dict_pkl/c_i_text.pkl", "rb").read() )
  xss = []
  yss = []

  f =  open("ku_teach.txt", "r") 
  lines = [line for line in f]
  #print( len(lines) )
  random.shuffle( lines )
  for fi, line in enumerate(lines):
    if fi >= 15000:
      break
    line = line.strip()
    head, tail = line.split("___SP___")
    if(len(head) > 50 or len(tail) > 50):
        continue

    xs = [ [0.]*128 for _ in range(50) ]
    ys = [ [0.]*128 for _ in range(50) ]
    
    for i, c in enumerate(head):
       if(c_i.get(c)):
         xs[i][c_i[c]] = 1.
    
    
    for i, c in enumerate(tail):
       if(c_i2.get(c)):
         ys[i][c_i2[c]] = 1.
    
    xss.append( np.array( list(reversed(xs)) ) )
    yss.append( np.array( list(reversed(ys)) ) )
    
  f.close()
        
  Xs = np.array( xss )
  Ys = np.array( yss )
  print(Xs.shape)
    

    #for i in range(2000):
  print_callback = LambdaCallback(on_epoch_end=callbacks)
  batch_size = random.randint( 32, 64 )
  random_optim = random.choice( [Adam(), SGD(), RMSprop()] )
  print( random_optim )
  autoencoder.optimizer = random_optim


  #traing
  keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')
  earlystop = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=10, verbose=0, mode='auto')
  tensorboard = keras.callbacks.TensorBoard(log_dir='logs', histogram_freq=0, write_graph=True)

  autoencoder.fit( Xs, Ys,  shuffle=True, batch_size=batch_size, epochs=100)
  now = str(time.time())

  autoencoder.save('model/'+ 'ku_epoc100' + '.h5')
  import gc; gc.collect()
  print("saved ..")
  print("logs...", buff )
    

In [ ]:
train()

(429, 50, 128)
Epoch 1/100
429/429 [==============================] - 35s - loss: 1.0047    
Epoch 2/100
429/429 [==============================] - 31s - loss: 1.0045    
Epoch 3/100
429/429 [==============================] - 33s - loss: 1.0043    
Epoch 4/100
106/429 [======>.......................] - ETA: 24s - loss: 0.9798

In [100]:
def predict():
  c_i = pickle.loads( open("dict_pkl/c_i_formula.pkl", "rb").read() )
  c_i2 = pickle.loads( open("dict_pkl/c_i_text.pkl", "rb").read() )
  i_c = { i:c for c, i in c_i.items() }
  i_c2 = { i:c for c, i in c_i2.items() }
  xss = []
  heads = []
  with open("testinp.txt", "r") as f: 
    for fi, line in enumerate(f):
      head = line.strip()
      #head, tail = line.split("___SP___")
      if(len(head)>50):
        break
      print(head)
      heads.append( head )
    
      xs = [ [0.]*128 for _ in range(50) ]
      for i, c in enumerate(head):
        if(c_i.get(c)):
         xs[i][c_i[c]] = 1.
      xss.append( np.array( list(reversed(xs)) ) )

  Xs = np.array( xss )
  print( Xs)
  #model =open("model/ku_first.h5")
  #print("loaded model is ", model)
  autoencoder.load_weights("model/ku_first.h5")

  Ys = autoencoder.predict( Xs ).tolist()
  for ez, (head, y) in enumerate(zip(heads, Ys)):
    terms = []
    for v in y:
      term = max( [(s, i_c2[i]) for i,s in enumerate(v)] , key=lambda x:x[0])[1]
      terms.append( term )
    tail = re.sub(r"」.*?$", "」", "".join( terms ) )
    print(ez, head, "___SP___", tail )

In [102]:
predict()

existsv.(_ギョウテン(v)&_スル(v)&Past(v))
[[[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]]
0 existsv.(_ギョウテン(v)&_スル(v)&Past(v)) ___SP___ 。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。。
